In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import sqlite3
import preprocesser
import drawing_chart
import query
import datetime
import plotly.offline as offline

In [2]:
shcode_list = query.query_shcode_list_today()

#2.target from csv
df = pd.read_csv("high_ratio_0.15_volume_money_30000000000_v2.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['interval_init','target_day','interval_end','shcode'])

In [3]:
for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    
    #interval calculate before 4 weeks , after 2 weeks
    interval_init_timestamp = target_day_list - datetime.timedelta(weeks=8)
    interval_start_timestamp = target_day_list - datetime.timedelta(weeks=4)
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=4)
    
    # make df using series(datetime to str)
    result = pd.DataFrame(dict(interval_init = interval_init_timestamp.strftime("%Y%m%d"),
                               interval_start = interval_start_timestamp.strftime("%Y%m%d"),
                               target_day = target_day_list.strftime("%Y%m%d"),
                               interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    
    del result['index']
    result['shcode'] = shcode
    
    fin_result = fin_result.append(result)

#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




#date to index
#df['date'] = pd.to_datetime(df['date'], format='%m%d%Y')
#df1 = df.set_index('date')


#group by 연습
#fin_result.groupby(['shcode'])['target_day'].count().sort_values(ascending = False).head()
#fin_result.groupby(['shcode','target_m']).filter(lambda x : (x['shcode'] == '014580').any())

In [4]:
#4. target month
#8월대상한정
#fin_result.groupby(['target_m']).count()
target_month = '201907'
target_fin_result = fin_result[fin_result['target_m']==target_month]
target_fin_result = target_fin_result.reset_index()

In [5]:
target_fin_result.head()

,index,interval_end,interval_init,interval_start,shcode,target_day,target_m,shname
0,0,20190826,20190603,20190701,000140,20190729,201907,하이트진로홀딩스
1,7,20190813,20190521,20190618,001820,20190716,201907,삼화콘덴서
2,8,20190822,20190530,20190627,002070,20190725,201907,남영비비안
3,9,20190823,20190531,20190628,002070,20190726,201907,남영비비안
4,32,20190801,20190509,20190606,004830,20190704,201907,덕성


In [6]:
#데이터추출
for i in range(len(target_fin_result['shcode'])):
    df = query.query_ohlcv(shcode = target_fin_result['shcode'].iloc[i],
                         fr=target_fin_result['interval_init'].iloc[i],
                         to=target_fin_result['interval_end'].iloc[i])
    df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d")
    #데이터전처리
    df = preprocesser.sma(df)
    df = preprocesser.vma(df)
    df = preprocesser.rsi(df)
    df = preprocesser.macd(df)
    df = preprocesser.stochastic(df)
    df = preprocesser.marking_cross_sma(df)
    df = preprocesser.marking_cross_rsi(df)
    df = preprocesser.marking_cross_macd(df)
    df = preprocesser.marking_cross_stochastic(df)
    fig, fig_go = drawing_chart.drawing_chart(df)
    # 제목붙이기
    fig['layout']['title'] = target_fin_result['shcode'].iloc[i] + '_' + \
                            target_fin_result['shname'].iloc[i] + '_' + \
                            target_fin_result['target_day'].iloc[i]

    savefolder = target_month
    file_name = target_fin_result['shcode'].iloc[i] + '_' + \
                target_fin_result['shname'].iloc[i] + '_' + \
                target_fin_result['target_day'].iloc[i] + '.html'
    #save
    offline.plot(fig, filename ='01.find_triger'+ '/' + savefolder + '/' + file_name, auto_open=False)
    
    print('1.종목코드: ' , target_fin_result['shcode'].iloc[i], 
          '2.종목명: ', target_fin_result['shname'].iloc[i], 
          '3.대상날짜: ', target_fin_result['target_day'].iloc[i])


1.종목코드:  000140 2.종목명:  하이트진로홀딩스 3.대상날짜:  20190729
1.종목코드:  001820 2.종목명:  삼화콘덴서 3.대상날짜:  20190716
1.종목코드:  002070 2.종목명:  남영비비안 3.대상날짜:  20190725
1.종목코드:  002070 2.종목명:  남영비비안 3.대상날짜:  20190726
1.종목코드:  004830 2.종목명:  덕성 3.대상날짜:  20190704
1.종목코드:  005360 2.종목명:  모나미 3.대상날짜:  20190708
1.종목코드:  005390 2.종목명:  신성통상 3.대상날짜:  20190708
1.종목코드:  006890 2.종목명:  태경화학 3.대상날짜:  20190723
1.종목코드:  008730 2.종목명:  율촌화학 3.대상날짜:  20190725
1.종목코드:  012610 2.종목명:  경인양행 3.대상날짜:  20190726
1.종목코드:  020120 2.종목명:  키다리스튜디오 3.대상날짜:  20190702
1.종목코드:  025560 2.종목명:  미래산업 3.대상날짜:  20190724
1.종목코드:  081000 2.종목명:  일진다이아 3.대상날짜:  20190718
1.종목코드:  093370 2.종목명:  후성 3.대상날짜:  20190703
1.종목코드:  096760 2.종목명:  JW홀딩스 3.대상날짜:  20190716
1.종목코드:  102280 2.종목명:  쌍방울 3.대상날짜:  20190730
1.종목코드:  138490 2.종목명:  코오롱플라스틱 3.대상날짜:  20190719
1.종목코드:  298690 2.종목명:  에어부산 3.대상날짜:  20190704
1.종목코드:  005290 2.종목명:  동진쎄미켐 3.대상날짜:  20190701
1.종목코드:  012700 2.종목명:  리드코프 3.대상날짜:  20190729
1.종목코드:  013310 2.종목명:  아진산업 3.대상날짜:  20190718
1.종

개별

In [9]:
#데이터추출
i = 6
df = query_ohlcv(shcode = fin_result['shcode'].iloc[i],
                 fr=fin_result['interval_init'].iloc[i],
                 to=fin_result['interval_end'].iloc[i])
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d")
#데이터전처리
df = preprocesser.sma(df)
df = preprocesser.vma(df)
df = preprocesser.rsi(df)
df = preprocesser.macd(df)
df = preprocesser.stochastic(df)
df = preprocesser.marking_cross_sma(df)
df = preprocesser.marking_cross_rsi(df)
df = preprocesser.marking_cross_macd(df)
df = preprocesser.marking_cross_stochastic(df)
fig, fig_go = drawing_chart.drawing_chart(df)

print('1.종목코드: ' , fin_result['shcode'].iloc[i], 
      '2.종목명: ', fin_result['shname'].iloc[i], 
      '3.대상날짜: ', fin_result['target_day'].iloc[i])
fig_go

In [ ]:
'''

fig['data'].append(dict(
                        x = [pd.to_datetime(fin_result['target_day'].iloc[i], format = "%Y-%m-%d") ] * len(df.close),
                        y = df.close, 
                        showlegend = False,
                        mode = 'lines', line =dict(width = 3), 
                        marker = dict(color = "rgb(255, 105, 180)"), 
                        yaxis = 'y'
                       ))

fig['layout']['shapes'] = dict({
    'type' : 'line',
    'x0': pd.to_datetime(fin_result['target_day'].iloc[i], format = "%Y-%m-%d"),
    'y0': 0,
    'x1': pd.to_datetime(fin_result['target_day'].iloc[i], format = "%Y-%m-%d"),
    'y1': 1,
    'line': {'color': "rgb(255, 105, 180)", 'width': 3},
})
'''

In [ ]:
#open
#offline.iplot(fig, filename="candlestick")